In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import re
from math import ceil
from time import sleep

In [ ]:
#FOR GOOGLE COLAB ONLY
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 958 kB 4.0 MB/s 
     |████████████████████████████████| 356 kB 52.1 MB/s 
     |████████████████████████████████| 138 kB 27.1 MB/s 
     |████████████████████████████████| 3.6 MB 48.6 MB/s 
     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: use options instead of chrome_options


In [ ]:
###FOR VISUAL CODE ONLY###
# from selenium import webdriver
# PATH = 'C:\Program Files (x86)\chromedriver.exe' 
# driver = webdriver.Chrome(PATH)

**We used Selenium to connect to the site and route between the relevant pages, when we reached the required page, we took the information using BeautifulSoup, and then exported the information to a csv file.**

In [ ]:
###Lists for DataFrame###
role = []
company = []
date = []
seniority = []
salary = []

In [ ]:
###Login###
driver.get("https://theworker.co.il")

get_in = driver.find_element_by_link_text('כניסה')
get_in.click()

sleep(0.5)

email = driver.find_element_by_id('ranker_email')
email.clear()
email.send_keys('adir4455@gmail.com')       

sleep(0.5)

password = driver.find_element_by_id('ranker_password')
password.clear()
password.send_keys('123456')

sleep(0.5)

login = driver.find_element_by_xpath('//*[@id="top"]/table/tbody/tr[2]/td/table/tbody/tr[2]/td[1]/form/table/tbody/tr[8]/td/input')
login.click()

sleep(0.5)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


KeyboardInterrupt: ignored

In [ ]:
###Functions###
def get_pages(driver):
    pages_html = driver.find_element_by_xpath('//*[@id="top"]/table/tbody/tr[2]/td/table/tbody/tr[2]/td[1]/table[1]/tbody/tr[2]/td[2]/span')
    pages_text = pages_html.text
    pages_text_to_array = pages_text.split()
    items_string = pages_text_to_array[2]
    items_int = int(items_string)
    pages = ceil(items_int/10)
    
    return pages
    

In [ ]:
base_url = 'https://theworker.co.il/salary/role/'

#Dict for routing between pages.
url_category = {'Software Student': 'סטודנט%20(תוכנה)', 
                'Cyber Security': 'אבטחת%20מידע',
                'Sysadmin Windows': 'sys%20admin%20win',
                'Database Administrator': 'DBA',
                'C Programmer': 'מתכנת%20C%2B%2B%20/%20C',
                'Java Programmer': 'מתכנת%20JAVA',
                '.NET': 'מתכנת%20NET.',
                'Team Leader': 'ראש%20צוות',
                'DevOps': 'DevOps',
                'Linux': 'LINUX%20/%20UNIX',
                'PHP': 'PHP',
                'Software Engineer': 'מהנדס%20תוכנה',
                'C# Programmer': 'מפתח%20C%23%0D%0A',
                'Mobile Programmer': 'מפתח%20Mobile',
                'Python Programmer': 'מפתח%20Python',
                'Web Programmer': 'מפתח%20WEB',
                'Algorithms Developer': 'מפתח%20אלגוריתמים%20(תוכנה)'}

In [ ]:

for key,value in url_category.items():
    url = base_url + f"{value}"
    driver.get(url)
    
    pages = get_pages(driver) #Get Pages
    
    ###Define Selectors (Next Page)###
    selector = '#top > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(2) > td.companies > table:nth-child(12) > tbody > tr > td > table > tbody > tr > td.nav_back > a > div'
    sub_selector = "#top > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(2) > td.companies > table:nth-child(13) > tbody > tr > td > table > tbody > tr > td.nav_back > a > div"
    
    #Itearate over all pages
    for page in range(pages):
      #Wait until selector loads.
        try:
            elem = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, selector)) #This is a dummy element
            )
            next_page = driver.find_element_by_css_selector(selector) 
        except:
            try:
                elem = WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, sub_selector)) #This is a dummy element
            )
                next_page = driver.find_element_by_css_selector(sub_selector)

                ###Swap selectors###
                selector, sub_selector = sub_selector, selector   
            except:
                driver.close()



    
    
        page_source = driver.page_source #Load html from driver.


        #Focusing on the relevant code areas on the page, inserting the data from the site into lists
        soup = BeautifulSoup(page_source, 'lxml') 
        soup_table = soup('table', attrs={"class":"nonselectable"})[0]
        soup_tr = soup_table('tr')

        for tr in soup_tr[2:]:
            data = (tr.get_text().split())
            if(data):
                data.pop()
                data.reverse()

                role.append(key)
                company.append(' '.join(data[3:]))
                date.append(data[2])
                seniority.append(data[1])
                salary.append(data[0])


        next_page.click() #Next page

In [ ]:
#Lists into raw data
raw_data = {'Role': role, 'Date':date, 'Seniority': seniority, 'Salary': salary, 'Company': company}

#Raw data into df
df = pd.DataFrame(raw_data)

#Export df to csv
# df.to_csv('high_tech_data.csv', encoding='utf-8-sig')
